# **CNN Strides - Rice Type Classification**

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d muratkokludataset/rice-image-dataset

Dataset URL: https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset
License(s): CC0-1.0
 95% 209M/219M [00:01<00:00, 210MB/s]
100% 219M/219M [00:01<00:00, 185MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/rice-image-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow
import keras
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

In [5]:
dataset_path = 'Rice_Image_Dataset'

# Loading and labeling images
images = []
labels = []

for subfolder in os.listdir(dataset_path):
    subfolder_path = os.path.join(dataset_path, subfolder)
    if not os.path.isdir(subfolder_path):
        continue
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        images.append(image_path)
        labels.append(subfolder)

In [6]:
df = pd.DataFrame({'image' : images, 'label': labels})

In [7]:
df.sample(5)

,image,label
17007,Rice_Image_Dataset/Basmati/basmati (2686).jpg,Basmati
59771,Rice_Image_Dataset/Jasmine/Jasmine (4588).jpg,Jasmine
74084,Rice_Image_Dataset/Ipsala/Ipsala (9348).jpg,Ipsala
50560,Rice_Image_Dataset/Jasmine/Jasmine (12935).jpg,Jasmine
71704,Rice_Image_Dataset/Ipsala/Ipsala (1574).jpg,Ipsala


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['image'], df['label'], test_size=0.2, random_state=42)

In [9]:
df_train = pd.DataFrame({'image' : X_train, 'label' : y_train})
df_test = pd.DataFrame({'image' : X_test, 'label' : y_test})

In [10]:
encode =  LabelEncoder()
y_train = encode.fit_transform(y_train)
y_test = encode.transform(y_test)

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                                height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
df_train['label'] = df_train['label'].astype(str)
df_test['label'] = df_test['label'].astype(str)

In [13]:
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='image',
    y_col='label',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='image',
    y_col='label',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical')

Found 60000 validated image filenames belonging to 5 classes.
Found 15000 validated image filenames belonging to 5 classes.


In [14]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(2**10, activation='relu'))
model.add(Dense(5, activation='softmax'))

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [16]:
history = model.fit(train_generator, epochs=5, validation_data=test_generator)

Epoch 1/5
1875/1875 [==============================] - 458s 243ms/step - loss: 0.2547 - accuracy: 0.9015 - val_loss: 0.0481 - val_accuracy: 0.9839
Epoch 2/5
1875/1875 [==============================] - 448s 239ms/step - loss: 0.0849 - accuracy: 0.9713 - val_loss: 0.0972 - val_accuracy: 0.9642
Epoch 3/5
1875/1875 [==============================] - 434s 232ms/step - loss: 0.0629 - accuracy: 0.9783 - val_loss: 0.2496 - val_accuracy: 0.9254
Epoch 4/5
1875/1875 [==============================] - 434s 232ms/step - loss: 0.0563 - accuracy: 0.9811 - val_loss: 0.2087 - val_accuracy: 0.9237
Epoch 5/5
1875/1875 [==============================] - 439s 234ms/step - loss: 0.0486 - accuracy: 0.9844 - val_loss: 0.7869 - val_accuracy: 0.8129
